In [1]:
import numpy as np 
import pandas as pd 
import os 
import matplotlib.pyplot as plt 
%matplotlib notebook 

In [2]:
def getMatrix(tname):
    basepath = './training_data/'+tname+'_data/'
    fnames = [os.path.join(basepath, x) for x in os.listdir(basepath) if x[0]!='.']
    out = None 
    print 'reading file =', 
    for i, fname in enumerate(fnames):
        print str(i+1)+',',
        if out is None: 
            out = pd.read_csv(fname, sep='\t', header=None, na_filter=False)
        else:
            out = pd.concat((out, pd.read_csv(fname, sep='\t', header=None, na_filter=False)))
    return out.values

In [3]:
x=getMatrix('order')

reading file = 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,


In [4]:
def tominutes(timestamps, toslots=False):
    out = np.zeros_like(timestamps, dtype=np.int32)
    for i, stamp in enumerate(timestamps):
        out[i]=int(stamp[-5:-3]) + (int(stamp[-8:-6])*60)
    return out if (toslots==False) else (out//10)+1

In [5]:
x[:,-1] = tominutes(x[:,-1], toslots=True)
#x[:,-1] = (x[:,-1]//10)+1

In [7]:
clustermap=pd.read_csv('./training_data/cluster_map/cluster_map', sep='\t', header=None)
clustermap=clustermap.values 

In [8]:
for cidx in xrange(clustermap.shape[0]):
    x[:,3][np.nonzero(x[:, 3]==clustermap[cidx,0])]=clustermap[cidx,1]

In [15]:
xpd=pd.DataFrame(x, columns=['orderid', 'driverid', 'userid', 'departure', 'destination', 
                        'price', 'timestamp'])

In [20]:
design=xpd.groupby(['departure', 'timestamp'])['driverid'].agg({'demand': 'count',
                                                        'supply': lambda x:np.sum(x!='NULL')})

In [22]:
design.to_csv('designMatrix.csv', sep=',')

In [23]:
design.shape

(9314, 2)